# Data Acquire

In [1]:
import time
import sched
import logging
import requests
from io import StringIO

import utils
# from database import upsert_bpa

import logging
import pymongo
import expiringdict

import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "SPY",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "5d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
data.reset_index(inplace=True,drop=False)
print(data)

[*********************100%***********************]  1 of 1 completed
                      Datetime        Open        High         Low  \
0    2020-11-30 09:30:00-05:00  362.829987  362.989990  362.769989   
1    2020-11-30 09:31:00-05:00  362.920013  363.010010  362.799988   
2    2020-11-30 09:32:00-05:00  362.899994  362.989990  362.829987   
3    2020-11-30 09:33:00-05:00  362.899994  363.019989  362.839996   
4    2020-11-30 09:34:00-05:00  362.970001  363.119995  362.730011   
...                        ...         ...         ...         ...   
1936 2020-12-04 15:55:00-05:00  369.459991  369.500000  369.269989   
1937 2020-12-04 15:56:00-05:00  369.309998  369.494995  369.304993   
1938 2020-12-04 15:57:00-05:00  369.438202  369.619995  369.429993   
1939 2020-12-04 15:58:00-05:00  369.605011  369.670013  369.510010   
1940 2020-12-04 15:59:00-05:00  369.665009  369.850006  369.579987   

           Close   Volume  
0     362.839996  1387460  
1     362.890015  1545880  
2     

In [11]:
# # different between Close & Adj Close
# sum = 0
# for i in range(1812):
#     sum += (data.Close[i] - data['Adj Close'][i])
# sum

0.0

In [3]:
# Download data for one stock
def download_one(t, p, i):
    data = yf.download(
            tickers = t,
            period = p,
            interval = i,
            group_by = 'ticker',
            auto_adjust = True,
            prepost = False,
            threads = True,
            proxy = None,
            # no progress bar
            progress=False)
    data.reset_index(inplace=True,drop=False)
    data.rename(columns = {'Close':t}, inplace = True) 
    data = data.drop(['Open', 'High','Low','Volume'], axis=1)
    return data

In [70]:
# arg
t = '^DJI'
p = '5d'
i = '1m'

In [71]:
download_one_test =  download_one(t,p,i)
print(download_one_test.shape)
download_one_test

(1939, 2)


,Datetime,^DJI
0,2020-11-30 09:30:00-05:00,29745.160156
1,2020-11-30 09:31:00-05:00,29733.519531
2,2020-11-30 09:32:00-05:00,29724.640625
3,2020-11-30 09:35:00-05:00,29714.259766
4,2020-11-30 09:36:00-05:00,29661.279297
...,...,...
1934,2020-12-04 15:55:00-05:00,30172.369141
1935,2020-12-04 15:56:00-05:00,30181.650391
1936,2020-12-04 15:57:00-05:00,30191.050781
1937,2020-12-04 15:58:00-05:00,30196.439453


In [49]:
# nan test
test1 = download_one('TSLA',p,i)
test2 = download_one('BTBT',p,i)
test3 = download_one('NDAQ',p,i)
testset1 = [test1, test2, test3]
# for i in testset1:
#     print(i.size)

# pd.concat([test1, test2], axis=1, sort=False, join = 'outer')
dft = pd.merge(test1, test3, on = 'Datetime', how = 'outer', sort = True)
dft

,Datetime,TSLA,NDAQ
0,2020-11-30 09:30:00-05:00,603.451172,127.970001
1,2020-11-30 09:31:00-05:00,605.719971,127.849998
2,2020-11-30 09:32:00-05:00,601.428711,127.334999
3,2020-11-30 09:33:00-05:00,598.429993,127.260002
4,2020-11-30 09:34:00-05:00,600.650024,NaN
...,...,...,...
1938,2020-12-04 15:55:00-05:00,597.950012,125.669998
1939,2020-12-04 15:56:00-05:00,598.685608,125.809998
1940,2020-12-04 15:57:00-05:00,598.750000,125.839996
1941,2020-12-04 15:58:00-05:00,598.679993,125.580002


In [50]:
dft.isna().sum()

Datetime      0
TSLA          2
NDAQ        143
dtype: int64

In [51]:
dft[100:1010]

,Datetime,TSLA,NDAQ
100,2020-11-30 11:11:00-05:00,581.869995,127.004997
101,2020-11-30 11:12:00-05:00,579.750000,126.959999
102,2020-11-30 11:13:00-05:00,580.612976,126.989998
103,2020-11-30 11:14:00-05:00,579.679993,126.900002
104,2020-11-30 11:15:00-05:00,579.200012,127.059998
...,...,...,...
1005,2020-12-02 13:17:00-05:00,564.046570,128.899994
1006,2020-12-02 13:18:00-05:00,563.530029,128.869995
1007,2020-12-02 13:19:00-05:00,562.599976,NaN
1008,2020-12-02 13:20:00-05:00,562.239685,128.880005


In [44]:
test1

,Datetime,TSLA
0,2020-11-30 09:30:00-05:00,603.451172
1,2020-11-30 09:31:00-05:00,605.719971
2,2020-11-30 09:32:00-05:00,601.428711
3,2020-11-30 09:33:00-05:00,598.429993
4,2020-11-30 09:34:00-05:00,600.650024
...,...,...
1936,2020-12-04 15:55:00-05:00,597.950012
1937,2020-12-04 15:56:00-05:00,598.685608
1938,2020-12-04 15:57:00-05:00,598.750000
1939,2020-12-04 15:58:00-05:00,598.679993


In [150]:
test2

,Datetime,BTBT
0,2020-11-30 09:30:00-05:00,7.1066
1,2020-11-30 09:31:00-05:00,7.0000
2,2020-11-30 09:32:00-05:00,6.7800
3,2020-11-30 09:33:00-05:00,6.8800
4,2020-11-30 09:34:00-05:00,6.9394
...,...,...
1420,2020-12-04 15:55:00-05:00,6.3299
1421,2020-12-04 15:56:00-05:00,6.3312
1422,2020-12-04 15:57:00-05:00,6.3500
1423,2020-12-04 15:58:00-05:00,6.3500


In [52]:
# # Download data for a portfolio
# def make_portfolio(port, p ,i):
#     one = [download_one(port[0],p,i)]
#     rst = []
#     for t in port[1:]:
#         one.append(download_one(t,p,i).drop(['Datetime'], axis=1))
#         rst = pd.concat(one, axis=1, sort=False)
#     return rst

In [45]:
# Download data for a portfolio
def make_portfolio(port, p ,i):
    rst = download_one(port[0],p,i)
    for t in port[1:]:
        one = download_one(t,p,i)
        rst = pd.merge(rst, one, on = 'Datetime', how = 'outer', sort = True)
    return rst

In [46]:
# test log

# - 12/5 11:24
# portfolio = ['SPY', 'SBUX', 'AAPL']

# - 12/6 3:10
# portfolio = ['TSLA', 'ENPH', 'ZM', 'MRNA', 'PTON', 'BTBT', 'TGT', 'WMT', 'SBUX', 'ABBV']



In [68]:
portfolio = ['TSLA', 'AMD', 'ZM', 'MRNA', 'PTON', 'AAPL', 'TGT', 'WMT', 'SBUX', 'ABBV', 'NIO', 'SPY', '^DJI']
make_portfolio(portfolio, p, i)

,Datetime,TSLA,AMD,ZM,MRNA,PTON,AAPL,TGT,WMT,SBUX,ABBV,NIO,SPY,^DJI
0,2020-11-30 09:30:00-05:00,603.451172,87.214996,480.513306,145.675003,110.349998,117.389999,179.509995,151.764999,97.900002,105.279999,54.040001,362.839996,29745.160156
1,2020-11-30 09:31:00-05:00,605.719971,87.680000,482.959991,148.190002,110.910004,117.589996,179.380005,151.705002,97.750000,105.410004,53.351700,362.890015,29733.519531
2,2020-11-30 09:32:00-05:00,601.428711,88.269997,484.265106,147.559998,110.750000,118.040001,179.020004,152.449997,97.650002,105.029999,52.869999,362.890015,29724.640625
3,2020-11-30 09:33:00-05:00,598.429993,88.550003,486.116394,146.527206,110.495003,118.087700,179.140106,151.770004,97.639999,105.127197,52.773998,362.970001,NaN
4,2020-11-30 09:34:00-05:00,600.650024,88.951599,483.480988,147.758194,110.834999,118.239998,178.919998,151.744995,97.605003,105.040001,53.029999,362.750000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,2020-12-04 15:55:00-05:00,597.950012,93.989998,409.943787,152.887405,114.639999,122.084999,174.660004,148.639999,102.114998,107.050003,43.020000,369.318207,30172.369141
1946,2020-12-04 15:56:00-05:00,598.685608,94.129997,410.399994,152.759995,114.570000,122.125000,174.750000,148.679993,102.059998,107.120003,43.025700,369.430115,30181.650391
1947,2020-12-04 15:57:00-05:00,598.750000,94.099998,410.250000,152.660004,114.470001,122.190002,175.000000,148.750000,102.105003,107.190002,43.020000,369.609985,30191.050781
1948,2020-12-04 15:58:00-05:00,598.679993,94.070000,410.160004,152.796005,114.540001,122.254997,174.929993,148.779999,102.224998,107.220001,42.990002,369.665009,30196.439453


In [135]:
portfolio = ['TSLA', 'ENPH', 'BTBT', 'NDAQ', 'SPY', '^DJI']
make_portfolio(portfolio, p, i)

,Datetime,TSLA,ENPH,BTBT,NDAQ,SPY,^DJI
0,2020-11-30 09:30:00-05:00,603.451172,142.440002,7.1066,127.970001,362.839996,29745.160156
1,2020-11-30 09:31:00-05:00,605.719971,142.910004,7.0000,127.849998,362.890015,29733.519531
2,2020-11-30 09:32:00-05:00,601.428711,144.710007,6.7800,127.334999,362.890015,29724.640625
3,2020-11-30 09:33:00-05:00,598.429993,143.414993,6.8800,127.260002,362.970001,29714.259766
4,2020-11-30 09:34:00-05:00,600.650024,144.154495,6.9394,127.080002,362.750000,29661.279297
...,...,...,...,...,...,...,...
1936,2020-12-04 15:55:00-05:00,597.950012,NaN,NaN,NaN,369.318207,30191.050781
1937,2020-12-04 15:56:00-05:00,598.685608,NaN,NaN,NaN,369.430115,30196.439453
1938,2020-12-04 15:57:00-05:00,598.750000,NaN,NaN,NaN,369.609985,30212.130859
1939,2020-12-04 15:58:00-05:00,598.679993,NaN,NaN,NaN,369.665009,NaN


In [69]:
# NaN with concat

rst = make_portfolio(portfolio, p, i)
rst #pandas.core.frame.DataFrame
rst.isna().sum()

Datetime     0
TSLA         9
AMD          9
ZM           9
MRNA         9
PTON         9
AAPL         9
TGT          9
WMT          8
SBUX         9
ABBV         8
NIO          9
SPY          9
^DJI        11
dtype: int64

In [48]:
# NaN with outer merge

rst = make_portfolio(portfolio, p, i)
rst #pandas.core.frame.DataFrame
rst.isna().sum()

Datetime      0
TSLA          9
ENPH         19
ZM            9
MRNA          9
PTON          9
BTBT        525
TGT           9
WMT           8
SBUX          9
ABBV          8
NDAQ        150
SPY           9
^DJI         11
dtype: int64

In [11]:
# Update once
def update_once(port, p ,i):
    df = make_portfolio(port, p, i)
    upsert_stock(df)

# Keep updating
def main_loop(timeout=DOWNLOAD_PERIOD):
    scheduler = sched.scheduler(time.time, time.sleep)

    def _worker():
        try:
            print("Updating")
            update_once(portfolio, p ,i)
        except Exception as e:
            logger.warning("main loop worker ignores exception and continues: {}".format(e))
        scheduler.enter(timeout, 1, _worker)    # schedule the next event

    scheduler.enter(0, 1, _worker)              # start the first event
    scheduler.run(blocking=True)

# Call main loop
if __name__ == '__main__':
    main_loop()

NameError: name 'DOWNLOAD_PERIOD' is not defined

In [13]:
def upsert_stock(df):
    """
    Update MongoDB database `stock` and collection `stock` with the given `DataFrame`.
    """
    db = client.get_database("stock")
    ####### collection #######
    collection = db.get_collection("stock3")
    update_count = 0
    for record in data.to_dict('records'):
        result = collection.replace_one(
            filter={'Datetime': record['Datetime']},    
            replacement=record,                         
            upsert=True)                                
        if result.matched_count > 0:
            update_count += 1
    logger.info("rows={}, update={}, ".format(df.shape[0], update_count) +
                "insert={}".format(df.shape[0]-update_count))

In [14]:
def download_data(retries=MAX_DOWNLOAD_ATTEMPT):
    """Returns BPA text from `BPA_SOURCE` that includes power loads and resources
    Returns None if network failed
    """
    text = None
    for i in range(retries):
        try:
            req = requests.get(url, timeout=0.5)
            req.raise_for_status()
            text = req.text
        except requests.exceptions.HTTPError as e:
            logger.warning("Retry on HTTP Error: {}".format(e))
    if text is None:
        logger.error('download_data too many FAILED attempts')
    return text

# Example project

In [64]:
BPA_SOURCE = "https://transmission.bpa.gov/business/operations/Wind/baltwg.txt"
MAX_DOWNLOAD_ATTEMPT = 5
DOWNLOAD_PERIOD = 10         # second
client = pymongo.MongoClient()
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'data.log')

In [65]:
def download_bpa(url=BPA_SOURCE, retries=MAX_DOWNLOAD_ATTEMPT):
    """Returns BPA text from `BPA_SOURCE` that includes power loads and resources
    Returns None if network failed
    """
    text = None
    for i in range(retries):
        try:
            req = requests.get(url, timeout=0.5)
            req.raise_for_status()
            text = req.text
        except requests.exceptions.HTTPError as e:
            logger.warning("Retry on HTTP Error: {}".format(e))
    if text is None:
        logger.error('download_bpa too many FAILED attempts')
    return text


def filter_bpa(text):
    """Converts `text` to `DataFrame`, removes empty lines and descriptions
    """
    # use StringIO to convert string to a readable buffer
    df = pd.read_csv(StringIO(text), skiprows=11, delimiter='\t')
    df.columns = df.columns.str.strip()             # remove space in columns name
    df['Datetime'] = pd.to_datetime(df['Date/Time'])
    df.drop(columns=['Date/Time'], axis=1, inplace=True)
    df.dropna(inplace=True)             # drop rows with empty cells
    return df

In [66]:
def upsert_bpa(df):
    """
    Update MongoDB database `energy` and collection `energy` with the given `DataFrame`.
    """
    db = client.get_database("energy")
    collection = db.get_collection("energy")
    update_count = 0
    for record in df.to_dict('records'):
        result = collection.replace_one(
            filter={'Datetime': record['Datetime']},    # locate the document if exists
            replacement=record,                         # latest document
            upsert=True)                                # update if exists, insert if not
        if result.matched_count > 0:
            update_count += 1
    logger.info("rows={}, update={}, ".format(df.shape[0], update_count) +
                "insert={}".format(df.shape[0]-update_count))

In [67]:
def update_once():
    t = download_bpa()
    df = filter_bpa(t)
    upsert_bpa(df)


def main_loop(timeout=DOWNLOAD_PERIOD):
    scheduler = sched.scheduler(time.time, time.sleep)

    def _worker():
        try:
            update_once()
        except Exception as e:
            logger.warning("main loop worker ignores exception and continues: {}".format(e))
        scheduler.enter(timeout, 1, _worker)    # schedule the next event

    scheduler.enter(0, 1, _worker)              # start the first event
    scheduler.run(blocking=True)


if __name__ == '__main__':
    main_loop()

2020-12-04 13:15:57,827 [upsert_bpa]: rows=1887, update=1851, insert=36
2020-12-04 13:16:10,305 [upsert_bpa]: rows=1888, update=1887, insert=1
2020-12-04 13:16:22,630 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:16:36,226 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:16:48,608 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:17:00,977 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:17:13,474 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:17:25,854 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:17:38,348 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:17:50,785 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:18:03,243 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:18:15,637 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:18:28,209 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-12-04 13:18:40,682 [upsert_bpa]: rows=1888, update=1888, insert=0
2020-

KeyboardInterrupt: 

#  Example Breakdown

In [18]:
text = None
for i in range(5):
    try:
        req = requests.get(BPA_SOURCE, timeout=0.5)
        req.raise_for_status()
        text = req.text
    except requests.exceptions.HTTPError as e:
        logger.warning("Retry on HTTP Error: {}".format(e))
if text is None:
    logger.error('download_bpa too many FAILED attempts')
text

"BPA Balancing Authority Load & Total Wind Generation\r\nat 5-minute intervals, last 7 days\r\nDates: 28Nov2020 - 05Dec2020 (last updated 4Dec2020 11:11:09) Pacific Time\r\nBased on 5-min MW readings from the BPA SCADA system for points 45583, 79687, 79682, 164377, 70681\r\n\r\nThis represents loads and resources in BPA's Balancing Authority (BA) including some that are not BPA's.\r\nIt does not include BPA loads served by transfer, scheduled out of region,\r\nor scheduled to customers with their own BAs such as Seattle and Tacoma\r\n\r\nBPA/Technical Operations (TOT-OpInfo@bpa.gov)\r\n\r\nDate/Time       \tLoad\tWind\tHydro\tFossil/Biomass\tNuclear\r\n11/28/2020 00:00\t5533\t1127\t6880\t345\t1167\r\n11/28/2020 00:05\t5547\t1146\t6674\t228\t1170\r\n11/28/2020 00:10\t5531\t1164\t6571\t228\t1167\r\n11/28/2020 00:15\t5529\t1208\t6500\t228\t1166\r\n11/28/2020 00:20\t5522\t1233\t6477\t225\t1167\r\n11/28/2020 00:25\t5528\t1257\t6451\t222\t1166\r\n11/28/2020 00:30\t5531\t1226\t6453\t226\t1167

In [24]:
df = pd.read_csv(StringIO(text), skiprows=11, delimiter='\t')
# df.columns = df.columns.str.strip()             # remove space in columns name
# df['Datetime'] = pd.to_datetime(df['Date/Time'])
# df.drop(columns=['Date/Time'], axis=1, inplace=True)
# df.dropna(inplace=True)             # drop rows with empty cells
df

,Date/Time,Load,Wind,Hydro,Fossil/Biomass,Nuclear
0,11/28/2020 00:00,5533.0,1127.0,6880.0,345.0,1167.0
1,11/28/2020 00:05,5547.0,1146.0,6674.0,228.0,1170.0
2,11/28/2020 00:10,5531.0,1164.0,6571.0,228.0,1167.0
3,11/28/2020 00:15,5529.0,1208.0,6500.0,228.0,1166.0
4,11/28/2020 00:20,5522.0,1233.0,6477.0,225.0,1167.0
...,...,...,...,...,...,...
2011,12/04/2020 23:35,NaN,NaN,NaN,NaN,NaN
2012,12/04/2020 23:40,NaN,NaN,NaN,NaN,NaN
2013,12/04/2020 23:45,NaN,NaN,NaN,NaN,NaN
2014,12/04/2020 23:50,NaN,NaN,NaN,NaN,NaN


In [52]:
type(df)

pandas.core.frame.DataFrame

In [81]:
df

,Date/Time,Load,Wind,Hydro,Fossil/Biomass,Nuclear
0,11/28/2020 00:00,5533.0,1127.0,6880.0,345.0,1167.0
1,11/28/2020 00:05,5547.0,1146.0,6674.0,228.0,1170.0
2,11/28/2020 00:10,5531.0,1164.0,6571.0,228.0,1167.0
3,11/28/2020 00:15,5529.0,1208.0,6500.0,228.0,1166.0
4,11/28/2020 00:20,5522.0,1233.0,6477.0,225.0,1167.0
...,...,...,...,...,...,...
2011,12/04/2020 23:35,NaN,NaN,NaN,NaN,NaN
2012,12/04/2020 23:40,NaN,NaN,NaN,NaN,NaN
2013,12/04/2020 23:45,NaN,NaN,NaN,NaN,NaN
2014,12/04/2020 23:50,NaN,NaN,NaN,NaN,NaN


# In One Block

In [72]:
import time
import sched
import logging
import requests
from io import StringIO

import utils
from database import upsert_bpa

import logging
import pymongo
import expiringdict

import pandas as pd
import yfinance as yf
import numpy as np


# parameter for testing
portfolio = ['TSLA', 'AMD', 'ZM', 'MRNA', 'PTON', 'AAPL', 'TGT', 'WMT', 'SBUX', 'ABBV', 'NIO', 'SPY', '^DJI']
p = '5d'
i = '1m'


# database info
MAX_DOWNLOAD_ATTEMPT = 5
DOWNLOAD_PERIOD = 10         # second
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'data.log')
client = pymongo.MongoClient()


# Download data for one stock
def download_one(t, p, i):
    data = yf.download(
            tickers = t,
            period = p,
            interval = i,
            group_by = 'ticker',
            auto_adjust = True,
            prepost = False,
            threads = True,
            proxy = None,
            # no progress bar
            progress=False)
    data.reset_index(inplace=True,drop=False)
    data.rename(columns = {'Close':t}, inplace = True) 
    data = data.drop(['Open', 'High','Low','Volume'], axis=1)
    return data


# Download data for a portfolio
def make_portfolio(port, p ,i):
    rst = download_one(port[0],p,i)
    for t in port[1:]:
        one = download_one(t,p,i)
        rst = pd.merge(rst, one, on = 'Datetime', how = 'outer', sort = True)
    return rst


# Update once
def update_once(port, p ,i):
    df = make_portfolio(port, p, i)
    upsert_stock(df)


# Keep updating
def main_loop(timeout=DOWNLOAD_PERIOD):
    scheduler = sched.scheduler(time.time, time.sleep)

    def _worker():
        try:
            print("Updating")
            update_once(portfolio, p ,i)
        except Exception as e:
            logger.warning("main loop worker ignores exception and continues: {}".format(e))
        scheduler.enter(timeout, 1, _worker)    # schedule the next event

    scheduler.enter(0, 1, _worker)              # start the first event
    scheduler.run(blocking=True)


# Call main loop
if __name__ == '__main__':
    main_loop()


Updating
2020-12-06 18:07:06,468 [upsert_stock]: rows=1950, update=1941, insert=9


KeyboardInterrupt: 